In [3]:
BATCH_SIZE = 128

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import string
import re
import os
import random
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
tf.config.run_functions_eagerly(True)

import tensorflow.keras as keras
from tensorflow.keras import layers

from collections import Counter


In [6]:
processed_data_location = "/content/drive/MyDrive/MachineLearning/NMT_English_to_Hindi/processed_data"

## Preprocessed data, vectorizers and embedding layers for english and hindi text

In [7]:
with open(processed_data_location + "/train.pkl",'rb') as f:
  train = pickle.load(f)

print("Number of Training examples:", len(train))

Number of Training examples: 1448877


In [8]:
random.seed(10)
random.shuffle(train)
train_size = len(train)
train, valid = train[:int(train_size*0.9)],train[int(train_size*0.9):]


In [9]:
def load_vectorizer(location):
  import pickle
  from_disk = pickle.load(open(location, "rb"))
  vectorizer = layers.TextVectorization.from_config(from_disk['config'])
  vectorizer.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
  vectorizer.set_weights(from_disk['weights'])
  return vectorizer

en_fasttext_vectorizer = load_vectorizer('/content/drive/MyDrive/MachineLearning/NMT_English_to_Hindi/EN_Fasttext_Vectorizer.pkl')
hi_fasttext_vectorizer = load_vectorizer('/content/drive/MyDrive/MachineLearning/NMT_English_to_Hindi/HI_Fasttext_Vectorizer.pkl')



In [10]:
def load_embedding(location):
  import pickle
  from_disk = pickle.load(open(location, "rb"))
  embedding_layer = layers.Embedding.from_config(from_disk['config'])

  return embedding_layer

en_embedding_layer = load_embedding("/content/drive/MyDrive/MachineLearning/NMT_English_to_Hindi/en_fasttext_embedding_layer.pkl")
hi_embedding_layer = load_embedding("/content/drive/MyDrive/MachineLearning/NMT_English_to_Hindi/hi_fasttext_embedding_layer.pkl")


## Tensorflow dataset



In [11]:
def create_tf_dataset(data):
  data = [(text['en'],text['hi']) for text in data]
  tf_data = tf.data.Dataset.from_tensor_slices(data)
  tf_data = tf_data.shuffle(BATCH_SIZE*4).batch(BATCH_SIZE).map(lambda X: (en_fasttext_vectorizer(X[:,0]),hi_fasttext_vectorizer(X[:,1])))
  tf_data = tf_data.map(lambda X_batch_en,X_batch_hi: ((X_batch_en,X_batch_hi[:,:-1]),X_batch_hi[:,1:]) )
  tf_data = tf_data.prefetch(4)

  return tf_data

In [12]:
training_dataset = create_tf_dataset(train)
validation_dataset = create_tf_dataset(valid)


/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [13]:
del train,valid

## Model

In [14]:
VOCAB_SIZE = hi_fasttext_vectorizer.vocabulary_size()
HIDDEN_UNITS = 1024

In [15]:
class Encoder(keras.layers.Layer):
  def __init__(self, encoder_units,embedding_layer,**kwargs):
    super().__init__(**kwargs)
    self.encoder_units = encoder_units
    self.embedding = embedding_layer
    self.lstm = keras.layers.LSTM(self.encoder_units,return_sequences=True,return_state = True,recurrent_initializer='glorot_uniform')

  def call(self,input_tokens):
    input_mask = tf.cast(input_tokens != 0, tf.float32)
    input_vectors = self.embedding(input_tokens)
    output,state_1,state_2 = self.lstm(input_vectors)
    states = [state_1,state_2]
    return output,states,input_mask

In [16]:
class Attention(keras.layers.Layer):
  def __init__(self, units, **kwargs):
    super().__init__(**kwargs)
    
    self.Attention_Matrix = keras.layers.TimeDistributed(keras.layers.Dense(units,kernel_initializer='glorot_uniform'))

  def call(self,encoded_input,decoded_output,input_mask=None):

    encoded_input = self.Attention_Matrix(encoded_input)

    if(input_mask!=None):
      input_mask = tf.expand_dims(input_mask, axis=-1)
      encoded_input = encoded_input*input_mask
    
    scores = tf.matmul(decoded_output,encoded_input,transpose_b = True)
    distribution = tf.nn.softmax(scores)
    output = tf.matmul(distribution, encoded_input)
    
    return output
  

In [17]:
class Decoder(keras.layers.Layer):
  def __init__(self, output_vocab_size, decoder_units, embedding_layer, **kwargs):
    super().__init__(**kwargs)

    self.output_vocab_size = output_vocab_size
    self.decoder_units = decoder_units
    
    self.embedding = embedding_layer
    self.lstm = keras.layers.LSTM(self.decoder_units,return_sequences=True,recurrent_initializer='glorot_uniform')
    
    self.attention = Attention(decoder_units)

    self.fc = keras.layers.TimeDistributed(keras.layers.Dense(self.output_vocab_size,kernel_initializer='glorot_uniform'))

  def call(self,input_tokens,encoded_input, state = None,input_mask = None):
    input_vectors = self.embedding(input_tokens)
    output_ = self.lstm(input_vectors,initial_state=state)
    attention_output_ = self.attention(encoded_input,output_,input_mask)
    output = self.fc(attention_output_)
    return output


In [18]:
class NMT(keras.Model):
  def __init__(self, output_vocab_size,encoder_units,decoder_units, encoder_embedding_layer, decoder_embedding_layer, **kwargs):
    super().__init__(**kwargs)
    self.encoder = Encoder(encoder_units, encoder_embedding_layer)
    self.decoder = Decoder(output_vocab_size, decoder_units, decoder_embedding_layer)

  def call(self,input_tokens):
    input_en_tokens,input_hi_tokens = input_tokens
    encoder_output, encoder_final_state,input_mask = self.encoder(input_en_tokens)
    output = self.decoder(input_hi_tokens, encoder_output,encoder_final_state,input_mask)
    return output


In [19]:
class MaskedLoss(tf.keras.losses.Loss):
  def __init__(self,**kwargs):
    super().__init__(**kwargs)
    self.name = 'masked_loss'
    self.loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')

  def __call__(self, y_true, y_pred,sample_weight):
    loss = self.loss(y_true, y_pred)
    mask = tf.cast(y_true != 0, tf.float32)
    loss *= mask

    return tf.reduce_mean(tf.reduce_sum(loss,axis = 1))

masked_loss = MaskedLoss()




In [20]:
NMT_model = NMT(VOCAB_SIZE,HIDDEN_UNITS,HIDDEN_UNITS,en_embedding_layer,hi_embedding_layer)

## Training

In [21]:
optimizer = keras.optimizers.Adam(1e-3)
NMT_model.compile(optimizer = optimizer,loss = masked_loss)

In [22]:
Early_Stopping = keras.callbacks.EarlyStopping(patience = 1,min_delta = 1,restore_best_weights=True)
Model_Checkpoint = keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/MachineLearning/NMT_English_to_Hindi/NMT_Models/LSTM_Attention.h5',save_best_only=True,save_weights_only=True)

In [23]:
# history_NMT = NMT_model.fit(training_dataset,epochs = 10,validation_data=validation_dataset,callbacks =[Early_Stopping,Model_Checkpoint,keras.callbacks.TerminateOnNaN()])


## Saving Encoder and Decoder for inference

In [30]:
for item in training_dataset.take(1):
  pass

In [31]:
_ = NMT_model(item[0])
NMT_model.load_weights('/content/drive/MyDrive/MachineLearning/NMT_English_to_Hindi/NMT_Models/LSTM_Attention.h5')

In [25]:
encoder = Encoder(HIDDEN_UNITS,en_embedding_layer)
_ = encoder(item[0][0])

decoder = Decoder(VOCAB_SIZE,HIDDEN_UNITS,hi_embedding_layer)
_ = decoder(item[0][1],_[0],_[1],_[2])

In [32]:
encoder.set_weights(NMT_model.get_weights()[:4])
decoder.set_weights(NMT_model.get_weights()[4:])

In [33]:
class EncoderModel(keras.Model):
  def __init__(self,encoder):
    super().__init__()
    self.encoder = encoder
  
  def call(self,input_tokens):
    return self.encoder(input_tokens)


class DecoderModel(keras.Model):
  def __init__(self,decoder):
    super().__init__()
    self.decoder = decoder
  
  def call(self,input_tokens,encoded_input, state = None,input_mask = None):
    return self.decoder(input_tokens,encoded_input, state = state,input_mask = input_mask)



In [34]:
Encoder_model = EncoderModel(encoder)
Decoder_model = DecoderModel(decoder)

In [35]:
_ = Encoder_model(item[0][0])
_ = Decoder_model(item[0][1],_[0],_[1],_[2])

In [ ]:
Encoder_model.save('/content/drive/MyDrive/MachineLearning/NMT_English_to_Hindi/NMT_Models/Encoder_LSTM_Attention_tf', save_format = 'tf',)
Decoder_model.save('/content/drive/MyDrive/MachineLearning/NMT_English_to_Hindi/NMT_Models/Decoder_LSTM_Attention_tf', save_format = 'tf',)
